In [753]:
%matplotlib inline

## ДЗ
Реализуйте задачу машинного перевода. Датасет: http://www.manythings.org/anki/

# Prepare data

In [754]:
# from io import open
import unicodedata
# import string
import re
import random

# import torch
# import torch.nn as nn
# from torch import optim
# import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [755]:
# my_lines

In [756]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [757]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Zа-яА-ЯёЁ.!?]+", r" ", s)
    return s

In [758]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('D:/Backup/Курс Нетология/Глубокое обучение/13 Text 2/dll_hw_text2/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    for p in pairs:
        del p[2]
    
    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [759]:
MAX_LENGTH = 9999999999999999

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [760]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'rus', True)
print(random.choice(pairs))

Reading lines...
Read 421765 sentence pairs
Trimmed to 29598 sentence pairs
Counting words...
Counted words:
rus 10848
eng 4456
['я почти закончил читать эту книгу .', 'i m almost finished reading this book .']


In [761]:
import pandas as pd

df = pd.DataFrame(pairs)
df.to_csv("D:/Backup/Курс Нетология/Глубокое обучение/13 Text 2/dll_hw_text2/pairs_tsv.tsv", sep="\t", index=False)

In [762]:
import torch
import torch.nn as nn
import torch.optim as optim

# from torchtext.legacy.datasets import Multi30k
from torchtext.data import Field, BucketIterator
import torchtext
from torchtext.data.utils import get_tokenizer

# These are the standard torch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim 
from torchtext import data # Helper libraries
import random
from tqdm import tqdm

import spacy
import numpy as np

import random
import math
import time

import copy

In [763]:
SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [764]:
spacy_ru = spacy.load('ru_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

In [765]:
def tokenize_ru(text):
    """
    Tokenizes Russian text from a string into a list of strings (tokens) and reverses it
    """
    return [tok.text for tok in spacy_ru.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [766]:
SRC = Field(tokenize = tokenize_ru, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

In [767]:
rus_pair_txt = 'D:/Backup/Курс Нетология/Глубокое обучение/7 RNN 3/dll_hw_7_rnn_3/rus.txt'
en_pair_txt = 'D:/Backup/Курс Нетология/Глубокое обучение/7 RNN 3/dll_hw_7_rnn_3/eng.txt'

In [768]:
MAX_VOCAB_SIZE = 30_000
MIN_COUNT = 3
MAX_SEQUENCE_LENGTH = 50
# BATCH_SIZE = 256 # Create Field object

# TEXT = data.Field(tokenize = 'spacy', lower=True, include_lengths = True, init_token = '<sos>', 
#             eos_token = '<eos>') # Specify Fields in our dataset

fields = [('input_sequence', SRC), ('output_sequence', TRG)] 

# .tsv containing the statement/response pairs
data_file = 'D:/Backup/Курс Нетология/Глубокое обучение/13 Text 2/dll_hw_text2/pairs_tsv.tsv' # Build the dataset

dialogue_data = data.TabularDataset(
    path=data_file, format='tsv',
    fields=fields)
    
# Build vocabulary, and include pretrained GLoVe vectors
SRC.build_vocab(dialogue_data,
                max_size=MAX_VOCAB_SIZE,
                min_freq=MIN_COUNT,
                vectors='glove.6B.300d',
                unk_init=torch.Tensor.normal_) # Split our dialogue data into training, validation, and test sets
train_data, test_data = dialogue_data.split()
train_data, valid_data = train_data.split()

In [769]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 14503
Number of validation examples: 6216
Number of testing examples: 8880


In [770]:
print(vars(train_data.examples[0]))

{'input_sequence': ['.', 'америки', 'из', 'старшеклассником', 'со', 'переписываюсь', 'я'], 'output_sequence': ['i', 'am', 'corresponding', 'with', 'an', 'american', 'high', 'school', 'student', '.']}


In [771]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [772]:
print(f"Unique tokens in source (ru) vocabulary: {len(SRC.vocab)}")
print(f"Unique tokens in target (en) vocabulary: {len(TRG.vocab)}")

Unique tokens in source (ru) vocabulary: 3601
Unique tokens in target (en) vocabulary: 2191


In [773]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [774]:
BATCH_SIZE = 133

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    sort=False,
    device = device)

In [775]:
#-------------------------------------

In [776]:
class Embedder(nn.Module):
    def __init__(self, vocab_size, d_model):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, d_model)
    def forward(self, x):
        return self.embed(x)

In [777]:
# class PositionalEncoder(nn.Module):
#     def __init__(self, d_model, max_seq_len = 80):
#         super().__init__()
#         self.d_model = d_model
        
#         # create constant 'pe' matrix with values dependant on 
#         # pos and i
#         pe = torch.zeros(max_seq_len, d_model)
#         for pos in range(max_seq_len):
#             for i in range(0, d_model, 2):
#                 pe[pos, i] = \
#                 math.sin(pos / (10000 ** ((2 * i)/d_model)))
#                 pe[pos, i + 1] = \
#                 math.cos(pos / (10000 ** ((2 * (i + 1))/d_model)))
                
#         pe = pe.unsqueeze(0)
#         self.register_buffer('pe', pe)
 
    
#     def forward(self, x):
#         # make embeddings relatively larger 
#         x = x * math.sqrt(self.d_model)
#         #add constant to embedding
#         seq_len = x.size(1)
#         x = x + torch.Tensor(self.pe[:,:seq_len]).cuda()
#         return x

In [778]:
class PositionalEncoder(torch.nn.Module):
    def __init__(self, d_model, max_seq_len=80):
        super().__init__()
        self.d_model = d_model
        pe = torch.zeros(max_seq_len, d_model)
        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = \
                    math.sin(pos / (10000 ** ((2 * i) / d_model)))
                pe[pos, i + 1] = \
                    math.cos(pos / (10000 ** ((2 * (i + 1)) / d_model)))
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)

    def forward(self, x):
        with torch.no_grad():
            x = x * math.sqrt(self.d_model)
            seq_len = x.size(1)
            pe = self.pe[:, :seq_len]
            x = x + pe
            return x

In [779]:
batch = next(iter(train_iterator))
input_seq = batch.input_sequence.transpose(0,1)
input_pad = SRC.vocab.stoi['<pad>']# creates mask with 0s wherever there is padding in the input
input_msk = (input_seq != input_pad).unsqueeze(1)

In [780]:
# create mask as before

target_seq = batch.output_sequence.transpose(0,1)
target_pad = TRG.vocab.stoi['<pad>']
target_msk = (target_seq != target_pad).unsqueeze(1)

size = target_seq.size(1) # get seq_len for matrix

nopeak_mask = np.triu(np.ones((1, size, size)),k=1).astype('uint8')
nopeak_mask = (torch.from_numpy(nopeak_mask) == 0).to(device)
target_msk = target_msk & nopeak_mask

In [781]:
def create_masks(input_seq, target_seq):
    input_pad = SRC.vocab.stoi['<pad>']# creates mask with 0s wherever there is padding in the input
    input_msk = (input_seq != input_pad).unsqueeze(1)
    
    target_seq = batch.output_sequence.transpose(0,1)
    target_pad = TRG.vocab.stoi['<pad>']
    target_msk = (target_seq != target_pad).unsqueeze(1)

    size = target_seq.size(1) # get seq_len for matrix

    nopeak_mask = np.triu(np.ones((1, size, size)),k=1).astype('uint8')
    nopeak_mask = (torch.from_numpy(nopeak_mask) == 0).to(device)
    target_msk = target_msk & nopeak_mask
    return input_msk.to(device), target_msk.to(device)

In [782]:
class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model, dropout = 0.1):
        super().__init__()
        
        self.d_model = d_model
        self.d_k = d_model // heads
        self.h = heads
        
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
        self.out = nn.Linear(d_model, d_model)
    
    def forward(self, q, k, v, mask=None):
        
        bs = q.size(0)
        
        # perform linear operation and split into h heads
        
        k = self.k_linear(k).view(bs, -1, self.h, self.d_k)
        q = self.q_linear(q).view(bs, -1, self.h, self.d_k)
        v = self.v_linear(v).view(bs, -1, self.h, self.d_k)
        
        # transpose to get dimensions bs * h * sl * d_model
       
        k = k.transpose(1,2)
        q = q.transpose(1,2)
        v = v.transpose(1,2)# calculate attention using function we will define next
        scores = attention(q, k, v, self.d_k, mask, self.dropout)
        
        # concatenate heads and put through final linear layer
        concat = scores.transpose(1,2).contiguous()\
        .view(bs, -1, self.d_model)
        
        output = self.out(concat)
    
        return output

In [834]:
def attention(q, k, v, d_k, mask=None, dropout=None):
    
    scores = torch.matmul(q, k.transpose(-2, -1)) /  math.sqrt(d_k)
    print(f'att_Scores {scores.size()}')
    if mask is not None:
        mask = mask.unsqueeze(1)
        print(f'unsq_Mask {mask.shape}')
        scores = scores.masked_fill(mask == 0, -1e9)
        scores = F.softmax(scores, dim=-1)
    
    if dropout is not None:
        scores = dropout(scores)
        
    output = torch.matmul(scores, v)
    return output

In [835]:
# def attention(query, key, value, d_k, mask=None, dropout=None):
#     "Compute 'Scaled Dot Product Attention'"
#     d_k = query.size(-1)
#     scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
#     if mask is not None:
#         mask = mask.unsqueeze(1)
#         scores = scores.masked_fill(mask == 0, -1e9)
#     p_attn = F.softmax(scores, dim = -1)
#     if dropout is not None:
#         p_attn = dropout(p_attn)
#     output = torch.matmul(p_attn, value)
#     return output

In [836]:
class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=2048, dropout = 0.1):
        super().__init__() 
        # We set d_ff as a default to 2048
        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)
    def forward(self, x):
        x = self.dropout(F.relu(self.linear_1(x)))
        x = self.linear_2(x)
        return x

In [837]:
class Norm(nn.Module):
    def __init__(self, d_model, eps = 1e-6):
        super().__init__()
    
        self.size = d_model
        # create two learnable parameters to calibrate normalisation
        self.alpha = nn.Parameter(torch.ones(self.size))
        self.bias = nn.Parameter(torch.zeros(self.size))
        self.eps = eps
    def forward(self, x):
        norm = self.alpha * (x - x.mean(dim=-1, keepdim=True)) \
        / (x.std(dim=-1, keepdim=True) + self.eps) + self.bias
        return norm

In [838]:
# build an encoder layer with one multi-head attention layer and one # feed-forward layer
class EncoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout = 0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.attn = MultiHeadAttention(heads, d_model)
        self.ff = FeedForward(d_model)
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)

    def forward(self, x, mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn(x2,x2,x2,mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.ff(x2))
        return x

    # build a decoder layer with two multi-head attention layers and
    # one feed-forward layer
class DecoderLayer(nn.Module):
    def __init__(self, d_model, heads, dropout=0.1):
        super().__init__()
        self.norm_1 = Norm(d_model)
        self.norm_2 = Norm(d_model)
        self.norm_3 = Norm(d_model)

        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)
        self.dropout_3 = nn.Dropout(dropout)

        self.attn_1 = MultiHeadAttention(heads, d_model)
        self.attn_2 = MultiHeadAttention(heads, d_model)
        self.ff = FeedForward(d_model).cuda()

    def forward(self, x, e_outputs, src_mask, trg_mask):
        x2 = self.norm_1(x)
        x = x + self.dropout_1(self.attn_1(x2, x2, x2, trg_mask))
        x2 = self.norm_2(x)
        x = x + self.dropout_2(self.attn_2(x2, e_outputs, e_outputs,
        src_mask))
        x2 = self.norm_3(x)
        x = x + self.dropout_3(self.ff(x2))
        return x# We can then build a convenient cloning function that can generate multiple layers:

def get_clones(module, N):
    return nn.ModuleList([copy.deepcopy(module) for i in range(N)])

In [839]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.layers = get_clones(EncoderLayer(d_model, heads), N)
        self.norm = Norm(d_model)
    def forward(self, src, mask):
        x = self.embed(src)
        x = self.pe(x)
        for i in range(N):
            x = self.layers[i](x, mask)
        return self.norm(x)
    
class Decoder(nn.Module):
    def __init__(self, vocab_size, d_model, N, heads):
        super().__init__()
        self.N = N
        self.embed = Embedder(vocab_size, d_model)
        self.pe = PositionalEncoder(d_model)
        self.layers = get_clones(DecoderLayer(d_model, heads), N)
        self.norm = Norm(d_model)
    def forward(self, trg, e_outputs, src_mask, trg_mask):
        x = self.embed(trg)
        x = self.pe(x)
        for i in range(self.N):
            x = self.layers[i](x, e_outputs, src_mask, trg_mask)
        return self.norm(x)

In [840]:
class Transformer(nn.Module):
    def __init__(self, src_vocab, trg_vocab, d_model, N, heads):
        super().__init__()
        self.encoder = Encoder(src_vocab, d_model, N, heads)
        self.decoder = Decoder(trg_vocab, d_model, N, heads)
        self.out = nn.Linear(d_model, trg_vocab)
    def forward(self, src, trg, src_mask, trg_mask):
        e_outputs = self.encoder(src, src_mask)
        d_output = self.decoder(trg, e_outputs, src_mask, trg_mask)
        output = self.out(d_output)
        return output# we don't perform softmax on the output as this will be handled 
# automatically by our loss function

In [841]:
d_model = 512
heads = 8
N = 6
src_vocab = len(SRC.vocab)
trg_vocab = len(TRG.vocab)

model = Transformer(src_vocab, trg_vocab, d_model, N, heads).to(device)

for p in model.parameters():
    if p.dim() > 1:
        nn.init.xavier_uniform_(p)# this code is very important! It initialises the parameters with a
# range of values that stops the signal fading or getting too big.
# See this blog for a mathematical explanation.optim = torch.optim.Adam(model.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

In [842]:
def train_model(epochs, print_every=100):
    
    model.train()
    
    start = time.time()
    temp = start
    
    total_loss = 0
    
    for epoch in range(epochs):
       
        for i, batch in enumerate(train_iterator):            
            
            src = batch.input_sequence.transpose(0,1)
            trg = batch.output_sequence.transpose(0,1)            
            
            # the French sentence we input has all words except
            # the last, as it is using each word to predict the next
            
            trg_input = trg[:, :-1]
            
            # the words we are trying to predict
            
            targets = trg[:, 1:].contiguous().view(-1)
            
            # create function to make masks using mask code above
            
            src_mask, trg_mask = create_masks(src, trg_input)
            
            preds = model(src, trg_input, src_mask, trg_mask)
            
            optim.zero_grad()
            
            loss = F.cross_entropy(preds.view(-1, preds.size(-1)),
            results, ignore_index=target_pad)            
            loss.backward()
            optim.step()
            
            total_loss += loss.data[0]
            if (i + 1) % print_every == 0:
                loss_avg = total_loss / print_every
                print("time = %dm, epoch %d, iter = %d, loss = %.3f, %ds per %d iters" % ((time.time() - start) // 60,
                epoch + 1, i + 1, loss_avg, time.time() - temp,
                print_every))
                total_loss = 0
                temp = time.time()

In [843]:
train_model(10, print_every=1)

att_Scores torch.Size([133, 8, 17, 17])
unsq_Mask torch.Size([133, 1, 1, 17])
att_Scores torch.Size([133, 8, 17, 17])
unsq_Mask torch.Size([133, 1, 1, 17])
att_Scores torch.Size([133, 8, 17, 17])
unsq_Mask torch.Size([133, 1, 1, 17])
att_Scores torch.Size([133, 8, 17, 17])
unsq_Mask torch.Size([133, 1, 1, 17])
att_Scores torch.Size([133, 8, 17, 17])
unsq_Mask torch.Size([133, 1, 1, 17])
att_Scores torch.Size([133, 8, 17, 17])
unsq_Mask torch.Size([133, 1, 1, 17])
att_Scores torch.Size([133, 8, 18, 18])
unsq_Mask torch.Size([133, 1, 15, 15])


RuntimeError: The size of tensor a (15) must match the size of tensor b (18) at non-singleton dimension 3

In [844]:
def translate(model, src, max_len = 80, custom_string=False):    
    model.eval()
    
    if custom_sentence == True:
        src = tokenize_ru(src)
        sentence = torch.LongTensor([[SRC.vocab.stoi[tok] for tok in sentence]]).cuda()
    
    src_mask = (src != input_pad).unsqueeze(-2)
    e_outputs = model.encoder(src, src_mask)
    
    outputs = torch.zeros(max_len).type_as(src.data)
    outputs[0] = torch.LongTensor([TRG.vocab.stoi['<sos>']])    
    
    for i in range(1, max_len):
        trg_mask = np.triu(np.ones((1, i, i),k=1).astype('uint8')
        trg_mask = (torch.from_numpy(trg_mask) == 0).cuda()

        out = model.out(model.decoder(outputs[:i].unsqueeze(0),
        e_outputs, src_mask, trg_mask))
        out = F.softmax(out, dim=-1)
        val, ix = out[:, -1].data.topk(1)

        outputs[i] = ix[0][0]
        if ix[0][0] == TRG.vocab.stoi['<eos>']:
            break
    return ' '.join([TRG.vocab.itos[ix] for ix in outputs[:i]])

SyntaxError: invalid syntax (<ipython-input-844-28c3f2a7fb36>, line 16)

https://towardsdatascience.com/how-to-code-the-transformer-in-pytorch-24db27c8f9ec